In [1]:
import ROOT
ROOT.TMVA.Tools.Instance()

Welcome to ROOTaaS 6.06/06


<ROOT.TMVA::Tools object at 0x57fee10>

In [2]:
outfileName = "tmvaTest.root"
outputFile = ROOT.TFile(outfileName,"RECREATE")

In [3]:
factory = ROOT.TMVA.Factory("TMVAClassification", outputFile,
                            "!V:!Silent:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

--- Factory                  : You are running ROOT Version: 6.06/06, Jul 6, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


In [4]:
factory.AddVariable("myvar1 := var1+var2", 'F')
factory.AddVariable( "myvar2 := var1-var2", "Expression 2", "", 'F' )
factory.AddVariable( "var3",                "Variable 3", "units", 'F' )
factory.AddVariable( "var4",                "Variable 4", "units", 'F' )
factory.AddSpectator( "spec1 := var1*2",  "Spectator 1", "units", -100.0, 100.0 )
factory.AddSpectator( "spec2 := var1*3",  "Spectator 2", "units", -100.0, 100.0 )

In [5]:
fname = "./tmva_class_example.root"
input = ROOT.TFile.Open( fname )
print("--- TMVAClassification       : Using input file: " + str(input.GetName()))

--- TMVAClassification       : Using input file: ./tmva_class_example.root


In [6]:
signal     = input.Get("TreeS")
background = input.Get("TreeB")
signalWeight     = 1.0
backgroundWeight = 1.0
factory.AddSignalTree    ( signal,     signalWeight     )
factory.AddBackgroundTree( background, backgroundWeight )
factory.SetBackgroundWeightExpression( "weight" )

--- DataSetInfo              : Added class "Signal"	 with internal class number 0
--- Factory                  : Add Tree TreeS of type Signal with 6000 events
--- DataSetInfo              : Added class "Background"	 with internal class number 1
--- Factory                  : Add Tree TreeB of type Background with 6000 events


In [7]:
mycuts = ROOT.TCut('')
mycutb = ROOT.TCut('')
factory.PrepareTrainingAndTestTree( mycuts, mycutb,
                                    "nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V" )

--- Factory                  : Preparing trees for training and testing...


In [8]:
# book different training methods

# Normal MLP
factory.BookMethod( ROOT.TMVA.Types.kMLP, "MLP", "H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )
# MLP with best learning method (BFGS)
factory.BookMethod( ROOT.TMVA.Types.kMLP, "MLPBFGS", "H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:TrainingMethod=BFGS:!UseRegulator" )
# MLP with best learning method (BFGS) and bayesian regulators
factory.BookMethod( ROOT.TMVA.Types.kMLP, "MLPBNN", "H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:TrainingMethod=BFGS:UseRegulator" )

<ROOT.TMVA::MethodMLP object ("MLPBNN") at 0x4594d60>

--- Factory                  : Booking method: MLP
--- MLP                      : Create Transformation "N" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm                     : Input : variable 'myvar1' (index=0).   <---> Output : variable 'myvar1' (index=0).
--- Norm                     : Input : variable 'myvar2' (index=1).   <---> Output : variable 'myvar2' (index=1).
--- Norm                     : Input : variable 'var3' (index=2).   <---> Output : variable 'var3' (index=2).
--- Norm                     : Input : variable 'var4' (index=3).   <---> Output : variable 'var4' (index=3).
--- MLP                      : Building Network
--- MLP                      : Initializing weights
--- Factory                  : Booking method: MLPBFGS
--- MLPBFGS                  : Create Transformation "N" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm                     : Inp

In [ ]:
# Train MVAs using the set of training events
factory.TrainAllMethods()
# ---- Evaluate all MVAs using the set of test events
factory.TestAllMethods()
# ----- Evaluate and compare performance of all configured MVAs
factory.EvaluateAllMethods()

In [ ]:
outputFile.Close()
print("==> Wrote root file: " + outputFile.GetName())
print("==> TMVAClassification is done!")
del factory
ROOT.TMVA.TMVAGui( outfileName )